![](https://storage.googleapis.com/gweb-developer-goog-blog-assets/images/Wagtail_Gemma_Blog_social_card_te.2e16d0ba.fill-1200x600.png)


# Hello
Gemma 2 will be fine-tunned using LoRA for greetings in different languages and cultures. In this notebook our focus will be **Hello** greeting. Keras will be used with Gemma 2.

# Installation

In [1]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!pip install -q -U kagglehub --upgrade

# Import libraries

In [2]:
import os
os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.
os.environ["JAX_PLATFORMS"] = ""
import keras
import keras_nlp
import kagglehub

import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

# Configuration

In [3]:
class Config:
    seed = 42
    dataset_path = "/kaggle/input/cultural-greetings"
    preset = "gemma2_2b_en" # name of pretrained Gemma 2
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    lora_rank = 4 # rank for LoRA, higher means more trainable parameters
    learning_rate=8e-5 # learning rate used in train
    epochs = 10 # number of epochs to train

In [4]:
keras.utils.set_random_seed(Config.seed)

# Load Data

In [5]:
df = pd.read_csv(f"{Config.dataset_path}/train_hello.csv")
df.head()

,Language,Culture,Greetings
0,English,Western,Hello
1,Spanish,Hispanic,Hola
2,French,French,Bonjour
3,German,Germanic,Hallo
4,Italian,Italian,Ciao


# Template Creation

In [6]:
template = "\n\nLanguage:\n{Language}\n\nCulture:\n{Culture}\n\nGreetings:\n{Greetings}"
df["prompt"] = df.apply(lambda row: template.format(Language=row.Language,
                                                             Culture=row.Culture,
                                                             Greetings=row.Greetings), axis=1)
data = df.prompt.tolist()

In [7]:
def colorize_text(text):
    for word, color in zip(["Language", "Culture", "Greetings"], ["blue", "red", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

In [8]:
gemma_causal_lm = keras_nlp.models.GemmaCausalLM.from_preset(Config.preset)

#The code creates a causal language model named gemma_causal_lm using a 
#pre-defined configuration specified in Config.preset. 

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [9]:
gemma_causal_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,614,341,888 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,614,341,888 (9.74 GB)

 Trainable params: 2,614,341,888 (9.74 GB)

 Non-trainable params: 0 (0.00 B)

# Gemma Cultural Greetings Class

In [10]:
class GemmaCG:
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.prompt = template
        self.gemma_causal_lm = gemma_causal_lm
        
    def query(self, Language, Culture):
        response = self.gemma_causal_lm.generate(
            self.prompt.format(
                Language=Language,
                Culture=Culture,
                Greetings=""), 
            max_length=self.max_length)
        display(Markdown(colorize_text(response)))

# Preprocessing

In [11]:
x, y, sample_weight = gemma_causal_lm.preprocessor(data[0:2])
print(x, y)

{'token_ids': Array([[    2,   109, 14357, ...,     0,     0,     0],
       [    2,   109, 14357, ...,     0,     0,     0]], dtype=int32), 'padding_mask': Array([[ True,  True,  True, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False]], dtype=bool)} [[   109  14357 235292 ...      0      0      0]
 [   109  14357 235292 ...      0      0      0]]


- `gemma_causal_lm.preprocessor(data[0:2])`: This line calls the `preprocessor` method of the `gemma_causal_lm` object, passing the first two elements of the `data` list as input. The `preprocessor` method is likely responsible for preprocessing the input data into a format suitable for the language model. It might involve tasks such as tokenization, normalization, and encoding.
- `x, y = gemma_causal_lm.preprocessor(data[0:2])`: This line unpacks the return value of the `preprocessor` method into two variables, `x` and `y`. The `x` variable likely represents the input features to the model, while the `y` variable represents the corresponding target labels.

# Fine-Tuning with LoRA

In [12]:
gemma_causal_lm.backbone.enable_lora(rank=Config.lora_rank)

- `gemma_causal_lm.backbone.enable_lora(rank=Config.lora_rank)`: This line calls the `enable_lora` method of the `backbone` attribute of the `gemma_causal_lm` object, passing the `rank` parameter with the value `Config.lora_rank`
- This code enables Low-Rank Adaptation (LoRA) for the `backbone` component of the `gemma_causal_lm` model. LoRA is a technique that allows for fine-tuning a pre-trained model on a smaller dataset while keeping the original model parameters mostly frozen. This can be beneficial in scenarios where the available training data is limited or the model is very large.
- The `rank` parameter specifies the rank of the low-rank matrices that will be introduced into the model. A higher rank generally allows for more flexibility but also increases the number of trainable parameters.

In [13]:
gemma_causal_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                                                  ┃                                   Config ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                              │                      Vocab size: 256,000 │
└───────────────────────────────────────────────────────────────┴──────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2304)        │   2,617,270,528 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     589,824,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,617,270,528 (9.75 GB)

 Trainable params: 2,928,640 (11.17 MB)

 Non-trainable params: 2,614,341,888 (9.74 GB)

# Train the Model

In [14]:
#set sequence length cf. config (512)
gemma_causal_lm.preprocessor.sequence_length = Config.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=Config.learning_rate),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_causal_lm.fit(data, epochs=Config.epochs, batch_size=Config.batch_size)

gemma_cg = GemmaCG()

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 129s 985ms/step - loss: 0.0959 - sparse_categorical_accuracy: 0.5788
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 98s 806ms/step - loss: 0.0249 - sparse_categorical_accuracy: 0.8503
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - loss: 0.0190 - sparse_categorical_accuracy: 0.8952
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - loss: 0.0175 - sparse_categorical_accuracy: 0.8998
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - loss: 0.0166 - sparse_categorical_accuracy: 0.9041
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - loss: 0.0147 - sparse_categorical_accuracy: 0.9100
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - loss: 0.0134 - sparse_categorical_accuracy: 0.9191
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - loss: 0.0125 - sparse_categorical_accuracy: 0.9298
Epoch 9/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 789ms/step - loss: 0.0115 - sparse_categorical_accuracy: 0.9395
Epoch 10/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 79s 78

# Known Data

In [15]:
df.iloc[1]

Language                                               Spanish
Culture                                               Hispanic
Greetings                                                 Hola
prompt       \n\nLanguage:\nSpanish\n\nCulture:\nHispanic\n...
Name: 1, dtype: object

In [16]:
row = df.iloc[1]
gemma_cg.query(row.Language,row.Culture)



**<font color='blue'>Language:</font>**
Spanish

**<font color='red'>Culture:</font>**
Hispanic

**<font color='green'>Greetings:</font>**
Hola

# New Data

In [17]:
Language = "Malay"
Culture = "Malay"
gemma_cg.query(Language,Culture)



**<font color='blue'>Language:</font>**
Malay

**<font color='red'>Culture:</font>**
Malay

**<font color='green'>Greetings:</font>**
Salam

In [18]:
preset_dir = ".\gemma2_2b_en_cultural_greetings"
gemma_causal_lm.save_to_preset(preset_dir)

# Acknowledgements:

Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

Marilia Prata https://www.kaggle.com/code/mpwolke/global-communication-gemma2keras